In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

username = os.getenv('METEOMATICS_USERNAME')
password = os.getenv('METEOMATICS_PASSWORD')
db_path = os.getenv("DB_PATH")


In [ ]:
"""
Nicosia, Cyprus
Latitude: 35.17465 | Longitude: 33.363878


Larnaca, Δήμος Λάρνακας, Cyprus
Latitude: 34.92361 | Longitude: 33.623618


Limassol, Limassol Municipality, Cyprus
Latitude: 34.68529 | Longitude: 33.033266
"""

In [3]:
import datetime as dt
import meteomatics.api as api

In [23]:
locations = {
    "limasol" : {'coords':[(34.68529,33.033266)]},
    "larnaca" : {'coords':[(34.92361,33.623618)]},
    "nicosia" : {'coords':[(35.17465,33.363878)]}
}

def get_weather_forecast(coord, name, save=False, delta_D = 7, interval_H=1):

    model = 'mix'
    startdate = dt.datetime.now(dt.timezone.utc).replace(minute=0, second=0, microsecond=0)
    delta = 7
    enddate = startdate + dt.timedelta(days=delta_D)
    interval = dt.timedelta(hours=interval_H)
    parameters = [ "t_2m:C", "precip_1h:mm", "prob_precip_1h:p", "wind_speed_10m:ms", "relative_humidity_2m:p", "frost_depth:cm", "sunshine_duration_1h:min", "global_rad:W" ]

    rename_mappings = {original : original.split(":")[0] for original in parameters}
    rename_mappings["validdate"]= "forecast_date"
    df = api.query_time_series(coord, startdate, enddate, interval,\
                               parameters, username, password, model=model)
    
    df = df.reset_index()   .drop(labels=['lat','lon'], axis=1)\
                            .rename(columns=rename_mappings)
    if save:
        df.to_csv(f"data/{name}_+{delta}Days.csv",index=False)
    
    return df


In [24]:
name = "limasol"
coords =  locations[name]['coords']
get_weather_forecast(coord=coords, name = name).head(5)

,forecast_date,t_2m,precip_1h,prob_precip_1h,wind_speed_10m,relative_humidity_2m,frost_depth,sunshine_duration_1h,global_rad
0,2025-03-01 12:00:00+00:00,17.4,0.0,1.0,2.0,55.1,NaN,46.1,457.2
1,2025-03-01 13:00:00+00:00,17.3,0.0,1.0,1.9,57.4,NaN,41.9,376.5
2,2025-03-01 14:00:00+00:00,17.1,0.0,1.0,1.9,57.5,NaN,43.3,236.7
3,2025-03-01 15:00:00+00:00,16.5,0.0,1.0,1.7,59.9,NaN,46.8,70.9
4,2025-03-01 16:00:00+00:00,15.4,0.0,1.0,0.9,66.3,NaN,32.9,0.0


In [25]:
def populate_dict():    
    for location_name, items in locations.items():
        coords =  items['coords']
        locations[location_name]['data']=get_weather_forecast(coord=coords, name = location_name,save=True)

populate_dict()


In [7]:
import sqlite3

def create_database():
    """Create the SQLite database and tables"""
    with open('schema.sql', 'r') as f:

        schema_sql = f.read()

    conn = sqlite3.connect(db_path)
    conn.executescript(schema_sql)

    conn.commit()
    conn.close()

    print(f"Database created at {db_path}")

create_database()

Database created at weather.db


In [8]:
def populate_location_tbl(db_path="weather.db", locations = locations):
    """
    Populate DB
    """
    # Sample locations - replace with your actual locations
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    for location_name, item in locations.items():
        coord = item['coords']
        cursor.execute("""
            INSERT INTO locations (name, latitude, longitude)
            VALUES (?, ?, ?)
        """, (
            location_name,
            coord[0][0],
            coord[0][1],
        ))
    
    conn.commit()
    conn.close()
    print(f"Loaded {len(locations)} locations into database")

populate_location_tbl()

Loaded 3 locations into database


In [26]:
conn = sqlite3.connect(db_path)

cursor = conn.cursor()
cursor.execute("SELECT name FROM locations")
location_names = [name[0] for name in cursor.fetchall()]
conn.close()

location_names


['limasol', 'larnaca', 'nicosia']

In [27]:
def add_dict_sql_mapping():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # For each location in the dictionary
    for location_name in locations.keys():
        # Convert location name to title case for consistent matching
        
        # Query the database for this location's ID
        cursor.execute("SELECT id FROM locations WHERE name = ?", (location_name,))
        result = cursor.fetchone()
        
        if result:
            # Add the ID to the location dictionary
            locations[location_name]['id'] = result[0]


    conn.close()

    return locations

locations = add_dict_sql_mapping()



In [28]:
locations['limasol']['data']

,forecast_date,t_2m,precip_1h,prob_precip_1h,wind_speed_10m,relative_humidity_2m,frost_depth,sunshine_duration_1h,global_rad
0,2025-03-01 12:00:00+00:00,17.4,0.0,1.0,2.0,55.1,NaN,46.1,457.2
1,2025-03-01 13:00:00+00:00,17.3,0.0,1.0,1.9,57.4,NaN,41.9,376.5
2,2025-03-01 14:00:00+00:00,17.1,0.0,1.0,1.9,57.5,NaN,43.3,236.7
3,2025-03-01 15:00:00+00:00,16.5,0.0,1.0,1.7,59.9,NaN,46.8,70.9
4,2025-03-01 16:00:00+00:00,15.4,0.0,1.0,0.9,66.3,NaN,32.9,0.0
...,...,...,...,...,...,...,...,...,...
164,2025-03-08 08:00:00+00:00,19.5,0.0,1.0,2.0,34.3,NaN,60.0,666.8
165,2025-03-08 09:00:00+00:00,20.3,0.0,1.0,2.4,35.8,NaN,60.0,763.3
166,2025-03-08 10:00:00+00:00,21.1,0.0,1.0,2.8,37.5,NaN,60.0,794.1
167,2025-03-08 11:00:00+00:00,21.7,0.0,1.0,3.3,39.4,NaN,60.0,757.2


In [ ]:
# location_id,forecast_date, t_2m, precip_1h, prob_precip_1h, wind_speed_10m, relative_humidity_2m, frost_depth, sunshine_duration_1h, global_rad 

In [29]:
import glob
import pandas as pd
def populate_forecast_tbl(db_path="weather.db", csv_dir="./data", locations= locations):
    """
    Load forecast data from CSV files
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM locations")
    location_names_db = [name[0] for name in cursor.fetchall()]  
    
    total_records = 0
    
    for location_name, items in locations.items():

        csv_path = f"data/{location_name}_+7Days.csv"
        
        
        if location_name not in location_names_db:
            print(f"Warning: Location '{location_name}' not found in database. Skipping file {csv_path}")
            continue
        
        location_id = locations[location_name]['id']
        
        # Load CSV data
        df = pd.read_csv(csv_path)

        df['location_id']=location_id
        print(df)
        # try:
        df.to_sql(
            'forecasts', 
            conn, 
            if_exists='append',  # Append to table if it exists
            index=False,        # Don't use DataFrame index
            chunksize=100,     # Process in chunks for better performance
            method='multi'      # Improves performance for multiple rows
        )
        # except:
        #     print('error')
        #     continue

        total_records += len(df)
    
    conn.close()
    print(f"Loaded {total_records} forecast records into database")

populate_forecast_tbl()

                 forecast_date  t_2m  precip_1h  prob_precip_1h  \
0    2025-03-01 12:00:00+00:00  17.4        0.0             1.0   
1    2025-03-01 13:00:00+00:00  17.3        0.0             1.0   
2    2025-03-01 14:00:00+00:00  17.1        0.0             1.0   
3    2025-03-01 15:00:00+00:00  16.5        0.0             1.0   
4    2025-03-01 16:00:00+00:00  15.4        0.0             1.0   
..                         ...   ...        ...             ...   
164  2025-03-08 08:00:00+00:00  19.5        0.0             1.0   
165  2025-03-08 09:00:00+00:00  20.3        0.0             1.0   
166  2025-03-08 10:00:00+00:00  21.1        0.0             1.0   
167  2025-03-08 11:00:00+00:00  21.7        0.0             1.0   
168  2025-03-08 12:00:00+00:00  22.4        0.0             1.0   

     wind_speed_10m  relative_humidity_2m  frost_depth  sunshine_duration_1h  \
0               2.0                  55.1          NaN                  46.1   
1               1.9                

In [159]:
conn = sqlite3.connect(db_path)


In [38]:
from db_utils import *

In [111]:

#List the latest forecast for each location for every day

sql_statement="""
    SELECT *
    FROM forecasts f
    INNER JOIN (
        SELECT DATE(forecast_date) AS day, MAX(forecast_date) AS max_time
        FROM forecasts
        GROUP BY DATE(forecast_date)
    ) sub ON DATE(f.forecast_date) = sub.day AND f.forecast_date = sub.max_time
    ORDER BY f.location_id;
"""
df = pd.read_sql(sql_statement,conn)
df

,id,location_id,forecast_date,t_2m,precip_1h,prob_precip_1h,wind_speed_10m,relative_humidity_2m,frost_depth,sunshine_duration_1h,global_rad,day,max_time
0,9,1,2025-03-02 23:00:00+00:00,12.4,0,1,1.8,80.3,None,0.0,0.0,2025-03-02,2025-03-02 23:00:00+00:00
1,33,1,2025-03-03 23:00:00+00:00,11.2,0,1,2.6,84.2,None,0.0,0.0,2025-03-03,2025-03-03 23:00:00+00:00
2,57,1,2025-03-04 23:00:00+00:00,16.7,0,1,8.4,44.4,None,0.0,0.0,2025-03-04,2025-03-04 23:00:00+00:00
3,81,1,2025-03-05 23:00:00+00:00,15.1,0,1,8.2,59.4,None,0.0,0.0,2025-03-05,2025-03-05 23:00:00+00:00
4,105,1,2025-03-06 23:00:00+00:00,10.7,0,1,2.0,73.5,None,0.0,0.0,2025-03-06,2025-03-06 23:00:00+00:00
5,129,1,2025-03-07 23:00:00+00:00,13.7,0,1,1.3,40.4,None,0.0,0.0,2025-03-07,2025-03-07 23:00:00+00:00
6,153,1,2025-03-08 23:00:00+00:00,13.6,0,1,1.3,71.4,None,0.0,0.0,2025-03-08,2025-03-08 23:00:00+00:00
7,169,1,2025-03-09 15:00:00+00:00,17.7,0,1,1.9,67.6,None,59.6,98.0,2025-03-09,2025-03-09 15:00:00+00:00
8,178,2,2025-03-02 23:00:00+00:00,12.7,0,1,2.6,85.7,None,0.0,0.0,2025-03-02,2025-03-02 23:00:00+00:00
9,202,2,2025-03-03 23:00:00+00:00,12.9,0,1,3.9,88.7,None,0.0,0.0,2025-03-03,2025-03-03 23:00:00+00:00


In [113]:
#List the average the_temp of the last 3 forecasts for each location for every day

sql_statement="""
WITH RankedRecords AS (
    SELECT 
        f.*,
        ROW_NUMBER() OVER (
            PARTITION BY DATE(forecast_date), location_id 
            ORDER BY forecast_date DESC
        ) AS row_num
    FROM forecasts f
)
SELECT * FROM RankedRecords
WHERE row_num <= 3
ORDER BY location_id, DATE(forecast_date), forecast_date DESC;
"""
df = pd.read_sql(sql_statement,conn)
df.loc[df['location_id']==1]


,id,location_id,forecast_date,t_2m,precip_1h,prob_precip_1h,wind_speed_10m,relative_humidity_2m,frost_depth,sunshine_duration_1h,global_rad,row_num
0,9,1,2025-03-02 23:00:00+00:00,12.4,0,1,1.8,80.3,None,0.0,0.0,1
1,8,1,2025-03-02 22:00:00+00:00,12.5,0,1,1.5,81.8,None,0.0,0.0,2
2,7,1,2025-03-02 21:00:00+00:00,12.6,0,1,1.1,82.7,None,0.0,0.0,3
3,33,1,2025-03-03 23:00:00+00:00,11.2,0,1,2.6,84.2,None,0.0,0.0,1
4,32,1,2025-03-03 22:00:00+00:00,11.4,0,1,2.7,82.3,None,0.0,0.0,2
5,31,1,2025-03-03 21:00:00+00:00,11.4,0,1,2.6,84.4,None,0.0,0.0,3
6,57,1,2025-03-04 23:00:00+00:00,16.7,0,1,8.4,44.4,None,0.0,0.0,1
7,56,1,2025-03-04 22:00:00+00:00,16.8,0,1,7.9,44.2,None,0.0,0.0,2
8,55,1,2025-03-04 21:00:00+00:00,16.9,0,1,7.4,44.0,None,0.0,0.0,3
9,81,1,2025-03-05 23:00:00+00:00,15.1,0,1,8.2,59.4,None,0.0,0.0,1


In [115]:
sql_statement="""
WITH RankedRecords AS (
    SELECT 
        DATE(forecast_date) AS day,
        location_id,
        t_2m,
        ROW_NUMBER() OVER (
            PARTITION BY DATE(forecast_date), location_id 
            ORDER BY forecast_date DESC
        ) AS row_num
    FROM forecasts
)
SELECT 
    day,
    location_id,
    AVG(t_2m) AS avg_t_2m
FROM RankedRecords
WHERE row_num <= 3
GROUP BY day, location_id
ORDER BY location_id, day;
"""
df = pd.read_sql(sql_statement,conn)
df

,day,location_id,avg_t_2m
0,2025-03-02,1,12.500000
1,2025-03-03,1,11.333333
2,2025-03-04,1,16.800000
3,2025-03-05,1,15.700000
4,2025-03-06,1,11.100000
5,2025-03-07,1,13.800000
6,2025-03-08,1,14.000000
7,2025-03-09,1,18.433333
8,2025-03-02,2,12.866667
9,2025-03-03,2,13.066667


In [144]:
sql_statement="PRAGMA table_info('forecasts');"
cursor = conn.cursor()

result = cursor.execute(sql_statement).fetchall()
# result

column_names = [column[1] for column in result[3:]]

column_names


['t_2m',
 'precip_1h',
 'prob_precip_1h',
 'wind_speed_10m',
 'relative_humidity_2m',
 'frost_depth',
 'sunshine_duration_1h',
 'global_rad']

In [169]:
#Get the top n locations based on each available metric where n is a parameter given to the API call.
n=2
cursor = conn.cursor()
for name in column_names:
    sql_statement=f"SELECT f.{name}, f.location_id, l.name, l.id FROM forecasts f, locations l WHERE f.location_id= l.id ORDER BY f.{name} DESC LIMIT {n};"
    result = cursor.execute(sql_statement).fetchall()
    # print(result)
pd.read_sql(sql_statement,conn).to_json()

'{"global_rad":{"0":797.7,"1":793.7},"location_id":{"0":3,"1":1},"name":{"0":"nicosia","1":"limasol"},"id":{"0":3,"1":1}}'

In [166]:
df = pd.read_sql("SELECT * FROM forecasts",conn)

df.sort_values(by='t_2m', ascending=False)

,id,location_id,forecast_date,t_2m,precip_1h,prob_precip_1h,wind_speed_10m,relative_humidity_2m,frost_depth,sunshine_duration_1h,global_rad
479,480,3,2025-03-08 12:00:00+00:00,24.8,0.0,1.0,2.3,20.6,None,60.0,651.2
286,287,2,2025-03-07 12:00:00+00:00,23.8,0.0,1.0,1.1,17.0,None,60.0,651.4
285,286,2,2025-03-07 11:00:00+00:00,23.6,0.0,1.0,2.4,18.7,None,60.0,755.9
455,456,3,2025-03-07 12:00:00+00:00,23.5,0.0,1.0,3.1,11.7,None,60.0,661.2
503,504,3,2025-03-09 12:00:00+00:00,23.5,0.0,1.0,2.2,28.6,None,59.5,631.9
...,...,...,...,...,...,...,...,...,...,...,...
470,471,3,2025-03-08 03:00:00+00:00,10.0,0.0,1.0,0.7,60.3,None,0.0,0.0
349,350,3,2025-03-03 02:00:00+00:00,9.7,0.0,1.0,0.4,98.6,None,0.0,0.0
350,351,3,2025-03-03 03:00:00+00:00,9.4,0.0,1.0,0.6,98.9,None,0.0,0.0
352,353,3,2025-03-03 05:00:00+00:00,9.2,0.0,1.0,0.6,97.5,None,9.9,51.2
